# Mixtral-8x7B

- [論文](https://arxiv.org/abs/2401.04088)
- [実装](https://github.com/mistralai/mistral-inference)
- [ウェブサイト](https://mistral.ai/news/mixtral-of-experts)

## 概要

**Mixtral 8x7B** は、スパース混合エキスパート（Sparse Mixture of Experts: **SMoE**）言語モデル

Mistral 7Bと異なり、各層が8つのフィードフォワードネットワーク（**エキスパート**）で構成される

トークン毎に **ルーターネットワーク** が2つのエキスパートを選択し、その出力を組み合わせて処理する

エキスパートを動的に選択する仕組みにより、47B（470億）のうち13B（130億）パラメータしか使用しない

計算コストとレイテンシを抑えて、Llama 2 70BやGPT-3.5に匹敵する性能を実現

SFT（Supervised Fine-Tuning）とDPO（Direct Preference Optimization）で訓練した **Mixtral 8x7B Instruct** も公開

## アーキテクチャ

Mixtralは、Mistral 7Bから以下の変更を加えている:

- Sliding Window Attention（SWA）からFully Dense Attentionに変更
- フィードフォワードブロックを混合エキスパート層（MoE層）に変更 

![](image/table_1.png)


### スパース混合エキスパート（Sparse Mixture of Experts）

混合エキスパートの詳細は別の[論文](https://arxiv.org/pdf/2401.04088)を参照

入力 $x$ に対するMoEの出力は、 **エキスパートネットワークの出力の重み付き和** で決まる

エキスパートネットワークの重みは、 **ゲーティングネットワークの出力** で決まる

スパース混合エキスパートの概要図:

![](image/figure_1.png)

$n$ 個のエキスパートネットワーク $\{E_0, E_1, ..., E_{n-1}\}$ が与えられたときの出力の重み付き和:

$$
\sum_{i=0}^{n-1}G(x)_{i}\cdot E_{i}(x).
$$

- $G(x)_i$: 入力 $x$ に対する $i$ 番目のエキスパートの重み（ゲーティングネットワーク）
- $E_i(x)$: 入力 $x$ を $i$ 番目のエキスパートが処理した出力（エキスパートネットワーク）

Mixtralのゲーティングネットワークは、上位 $K$ 個（Top-K）のロジットに対してソフトマックスを適用した関数:

$$
G(x) := \text{Softmax}(\text{TopK}(x \cdot W_g))
$$

- $W_g$: ゲーティングネットワークの重み行列
- $x\cdot W_g$: 各エキスパートのスコア（ロジット）
- $\text{TopK}$: 上位$K$個に含まれないロジットをマイナス無限大にする関数
- $\text{Softmax}(\cdot)$: マイナス無限大になったロジットを除いて合計 $1.0$ の確率分布に変換する関数

使用するエキスパート数 $K$ を固定し、エキスパートの総数 $n$ を増やすことで、効率的にパラメータ総数を増加できる:

- トークン毎に使用するパラメータ数を **アクティブパラメータ数** （active parameter count）と呼ぶ
- モデルのパラメータ総数を **スパースパラメータ数** （sparse parameter count）と呼ぶ

MoE層は、単一もしくは複数のGPUで効率的に実行できる:

- 単一GPUでの効率化手法
    - [Megablocks][1]: MoEのFFNの操作を大きなスパース行列乗算として扱い、実行速度を向上させる
- 複数GPUでの効率化手法
    - モデル並列化（Model Parallelism techniques）: モデルを層ごとに分けて複数のGPUに展開する
    - [エキスパート並列化][2]（Expert Parallelism: EP）: エキスパートをグループに分けて複数のGPUに展開する

[1]: https://proceedings.mlsys.org/paper_files/paper/2023/hash/5a54f79333768effe7e8927bcccffe40-Abstract-mlsys2023.html
[2]: https://arxiv.org/abs/1701.06538


Mixtralでは、エキスパートをSwiGLUアーキテクチャで実装し、使用するエキスパート数を $K=2$ とする:

$$
y = \sum_{i=0}^{n-1} \text{Softmax}(\text{Top2}(x\cdot W_g))_i \cdot \text{SwiGLU}_i(x)
$$

- $\text{Softmax}(\text{Top2}(x\cdot W_g))_i$: $i$ 番目のエキスパートに対する重み
- $\text{SwiGLU}_i(x)$: $i$ 番目のエキスパートの出力

## ベンチマーク結果

MixtralとLlamaをベンチマークで評価し比較:

- 常識推論（0-shot）
    - Hellaswag: 文脈から自然に続く結末を選ぶ
    - Winogrande: 代名詞が指している単語を選ぶ
    - PIQA（Physical Interaction Question Answering）: 物理的な理解が必要な選択肢を選ぶ
    - SIQA（Social Interaction QA）:人の感情の理解が必要な選択肢を選ぶ
    - OpenbookQA: 一般的な科学的事実（Open Book）の理解が必要な選択肢を選ぶ
    - ARC-Easy（AI2 Reasoning Challenge）: 小学生レベルの科学の理解が必要な選択肢を選ぶ
    - ARC-Challenge: 中学生レベルの科学の理解が必要な選択肢を選ぶ
    - CommonsenseQA: 社会常識の理解が必要な選択肢を選ぶ
- 世界知識（5-shot）
    - NaturalQuestions: 与えられたWikipediaのページから長い回答と短い回答を抽出する
    - TriviaQA: ウェブページやWikipediaのページが与えられ、それらを統合する必要のある回答を抽出する
- 読解（0-shot）
    - BoolQ: 与えられた文章に対して、はい/いいえで回答する
    - QuAC（Question Answering in Context）: 一人のユーザーが連続して質問しそれに対して回答し続ける
- 数学
    - GSM8K（8-shot）: 小学生レベルの算数問題
    - MATH（4-shot）: 競技数学レベルの難しい数学問題
- コード
    - Humaneval（0-shot）: 人が作成したPython関数をヒントから完成させる
    - MBPP（Mostly Basic Python Programming）（3-shot）: 初心者向けの基本的なPythonプログラミング問題を解く
- 総合
    - MMLU（Massive Multitask Language Understanding）（5-shot）: 57の異なる分野の選択問題
    - BBH（Big-Bench Hard）（3-shot）: 現在のモデルが苦手とする23の挑戦的なタスク
    - AGI Eval（3-5-shot）: 米国の大学入学試験、法科大学院試験、医師国家試験などの選択問題

Mixtralは、アクティブパラメータ数が5倍多いLlama2 70Bを多くのベンチマークで上回った（コードと数学が強い）:

![](image/figure_2.png)

![](image/table_2.png)

Mixtralは、アクティブパラメータ数が少なく性能が高い:

![](image/figure_3.png)

Mixtralは、LlaMA 2 70Bより優れ、GPT-3.5（GPT-3.5-Turbo）に匹敵する性能を示した:

![](image/table_3.png)

Mixtralは、英語の他にフランス語・ドイツ語・スペイン語・イタリア語でLlama2 70 Bを大幅に上回る性能:

![](image/table_4.png)

長い文章から探し出すタスク（[passkey retrieval][1]）では、100%の検索性能を示し、困惑度（perplexity）も長さに応じて減少:

![](image/figure_4.png)

[1]: https://arxiv.org/abs/2305.16300

Llama 2と比較して、BBQ（Bias Benchmark for QA）で低いバイアスを示した:

![](image/figure_5.png)

指示チューニング済みモデルは、MT-Benchではオープンウェイトモデルの中で最も高い:

![](image/figure_6.png)

## ルーティング分析

The Pileの検証データセットを使い、トピック毎に0層目・15層目・31層目のエキスパート選択状態を測定

トピックに基づいたエキスパートの選択に明確なパターンは見られなかった（数学のみわずかに反応）:

![](image/figure_7.png)

![](image/table_5.png)

トークンごとのエキスパートの割当では、`self`・`Question`・インデント・連続したトークンが同じルーティング:

![](image/figure_8.png)

## 実装

In [1]:
%pip install -qU transformers==4.57.1
%pip install sentencepiece protobuf

try:
    import google.colab
except ImportError:
    from dotenv import load_dotenv
    import os
    load_dotenv()
    HF_TOKEN = os.getenv("HF_TOKEN")

assert HF_TOKEN

import os
import logging as logging_
import transformers
from transformers import PretrainedConfig
from transformers.utils import logging

from typing import Callable, Optional, Union

import torch
from torch import nn
import numpy as np

# トークナイザー

import json
import os
import unicodedata
from functools import lru_cache
from typing import Optional

import regex as re

from transformers.tokenization_utils import AddedToken, PreTrainedTokenizer
from transformers.utils import logging

# モデル

from typing import Callable, Optional, Union

import torch
from torch import nn

assert torch.cuda.is_available(), "CUDAを使用できません"

from transformers.activations import ACT2FN
from transformers.cache_utils import Cache, DynamicCache
from transformers.generation import GenerationMixin
from transformers.integrations import use_kernel_forward_from_hub
from transformers.masking_utils import create_causal_mask, create_sliding_window_causal_mask
from transformers.modeling_flash_attention_utils import FlashAttentionKwargs
from transformers.modeling_layers import (
    GenericForQuestionAnswering,
    GenericForSequenceClassification,
    GenericForTokenClassification,
    GradientCheckpointingLayer,
)
from transformers.modeling_outputs import BaseModelOutputWithPast, CausalLMOutputWithPast
from transformers.modeling_rope_utils import ROPE_INIT_FUNCTIONS, dynamic_rope_update
from transformers.modeling_utils import ALL_ATTENTION_FUNCTIONS, PreTrainedModel
from transformers.processing_utils import Unpack
from transformers.utils import TransformersKwargs, auto_docstring, can_return_tuple
from transformers.utils.deprecation import deprecate_kwarg
from transformers.utils.generic import check_model_inputs
from transformers.models.qwen2.configuration_qwen2 import Qwen2Config


# デバイス設定

torch.set_default_device("cuda")

# ログ設定

if os.path.exists('debug.log'):
    os.remove('debug.log')

def custom_format(record):
    match record.levelno:
        case logging_.DEBUG:
            level = '🟦'
        case logging_.INFO:
            level = '🟩'
        case logging_.WARNING:
            level = '🟨'
        case logging_.ERROR:
            level = '🟥'
        case logging_.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logging.set_verbosity_debug()
logger = logging.get_logger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging_.Formatter()
formatter.format = custom_format

file_handler = logging_.FileHandler('debug.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging_.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.info(f"Transformers version: {transformers.__version__}")
logger.info(f"Numpy version: {np.__version__}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/opt/miniconda/envs/py312/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


AssertionError: CUDAを使用できません